In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
440,2024-09-24,Europa Eurocopa,13:30,Turk Telekom,Lietkabelis,1.19,4.37,159.5,1.85,1.85,-10.5,1.94,1.77,8pINGxFd,0.840336,0.228833,0.540541,0.540541,0.069169,0.4,0.4,NaN,NaN,178.716,135.915372,0.760510,239.514,79.650382,0.332550,136.718,252.334,127.36,233.60,0.0,0.0,0.0,0.0,0.808849,0.000000,0.064802,0.02,0.004,47.500000,0.0,0.0,0.0,-5.00,-1.000,-3.370000,0.0,0.0,0.0
441,2024-09-24,Europa Eurocopa,14:30,Hamburg,JL Bourg,2.65,1.44,163.5,1.87,1.83,4.5,1.92,1.79,AkzyQthA,0.377358,0.694444,0.534759,0.546448,0.071803,0.8,0.4,NaN,NaN,252.306,84.936858,0.336642,137.818,36.951649,0.268119,237.234,128.862,225.76,112.23,0.0,0.0,0.0,0.0,0.418386,0.015289,0.049555,-5.00,-1.000,-1.650000,0.0,0.0,0.0,0.17,0.034,12.941176,0.0,0.0,0.0
442,2024-09-24,Europa Eurocopa,15:30,Joventut,Hapoel Tel-Aviv,2.53,1.48,168.5,1.85,1.85,3.5,2.00,1.72,82BgD384,0.395257,0.675676,0.540541,0.540541,0.070933,0.8,0.4,NaN,NaN,183.544,76.982702,0.419424,181.922,49.807640,0.273786,140.672,159.034,119.00,173.90,0.0,0.0,0.0,0.0,0.370305,0.000000,0.106446,-1.12,-0.224,-6.830357,0.0,0.0,0.0,2.29,0.458,1.048035,0.0,0.0,0.0
443,2024-09-24,Europa Eurocopa,16:00,Gran Canaria,Trento,1.14,5.24,163.5,1.83,1.87,-12.5,2.05,1.69,MTgt4aVj,0.877193,0.190840,0.546448,0.534759,0.068033,0.8,0.6,NaN,NaN,112.260,19.551522,0.174163,170.882,58.580223,0.342811,103.532,288.256,133.20,200.97,0.0,0.0,0.0,0.0,0.908821,0.015289,0.136128,-2.72,-0.544,-0.257353,0.0,0.0,0.0,0.93,0.186,22.795699,0.0,0.0,0.0
444,2024-09-24,Dinamarca Liga De Basquete,14:00,Værløse,Bears Academy,1.64,2.07,160.5,1.83,1.83,-3.5,1.93,1.74,fHnXO1pA,0.609756,0.483092,0.546448,0.546448,0.092848,0.8,0.0,NaN,NaN,302.620,217.761947,0.719589,520.174,527.696310,1.014461,366.508,0.000,107.91,693.00,0.0,1.0,0.0,0.0,0.163912,0.000000,0.073215,-2.78,-0.556,-1.151079,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
445,2024-09-24,Eua Wnba,20:30,New York Liberty F,Atlanta Dream F,1.12,6.40,155.5,1.81,1.93,-13.5,2.02,1.77,2y3Hdm9T,0.892857,0.156250,0.552486,0.518135,0.049107,0.0,0.6,NaN,NaN,127.820,26.779891,0.209513,211.260,103.081556,0.487937,0.000,197.502,95.45,393.30,0.0,0.0,0.0,0.0,0.992958,0.045376,0.093286,0.00,0.000,inf,0.0,0.0,0.0,-1.52,-0.304,-17.763158,0.0,0.0,0.0
446,2024-09-24,Eua Wnba,22:30,Las Vegas Aces F,Seattle Storm F,1.24,4.15,160.5,1.89,1.89,-10.5,2.02,1.70,tr2Ff0wq,0.806452,0.240964,0.529101,0.529101,0.047415,0.0,0.8,NaN,NaN,132.534,24.312902,0.183447,175.112,43.439936,0.248069,0.000,178.230,101.40,240.53,0.0,0.0,0.0,0.0,0.763518,0.000000,0.121653,0.00,0.000,inf,0.0,0.0,0.0,-3.30,-0.660,-4.772727,0.0,0.0,0.0
447,2024-09-24,Europa Liga Da Aba 2,13:00,MZT Skopje,Tivat,1.38,2.70,150.5,1.85,1.81,-6.5,1.94,1.74,EwcudDwo,0.724638,0.370370,0.540541,0.552486,0.095008,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.457540,0.015456,0.076859,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
448,2024-09-24,Europa Euroliga Feminina,15:00,Zaragoza F,CSM Constanta F,1.02,25.00,134.5,1.88,1.78,-19.5,1.90,1.80,QmoWKn1M,0.980392,0.040000,0.531915,0.561798,0.020392,0.4,0.0,NaN,NaN,136.002,26.287026,0.193284,NaN,NaN,NaN,110.702,0.000,122.55,0.00,0.0,0.0,0.0,0.0,1.303337,0.038640,0.038222,3.43,0.686,0.029155,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
449,2024-09-24,França Copa Da França,15:00,Angers,Orleans,4.49,1.15,161.5,1.83,1.83,9.5,1.95,1.72,x2HwpySQ,0.222717,0.869565,0.546448,0.546448,0.092282,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
